<a href="https://colab.research.google.com/github/Gajendra-theDataEngineer004627/All_Learnings/blob/main/Final_Project_WebScrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
#---------------------------------Extracting Name of the Product---------------------------#
# Function to extract Product Title
def get_title(soup):
    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        # Inner NavigatableString Object
        title_value = title.text
        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""
    return title_string

#--------------------------------Extracting Product Price----------------------------------#
# Function to extract Product Price
def get_price(soup):
    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()
    except AttributeError:
        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()
        except:
            price = ""
    return price

#-------------------------------Extracting Ratings -----------------------#
# Function to extract Product Rating
def get_rating(soup):
    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""
    return rating


#------------------------------------Extracting Number of users reviewed -------------#
# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()
    except AttributeError:
        review_count = ""

    return review_count

#----------------------------------Extract Availability Status ----------------#

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

In [21]:
if __name__ == '__main__':

    # add your user agent
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    url = "https://www.amazon.in/s?k=mobiles&ref=nb_sb_noss"

    # HTTP Request
    webpage = requests.get(url, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))



# Assuming you already have a dictionary 'd'
        amazon_df = pd.DataFrame.from_dict(d)

# Replace empty titles with NaN in the 'title' column
        amazon_df['title'].replace('', np.nan, inplace=True)

# Drop rows with NaN values in the 'title' column
        amazon_df.dropna(subset=['title'], inplace=True)

# Sort the DataFrame by 'reviews' column in descending order
        final_amazon_df = amazon_df.sort_values(by='reviews', ascending=False)

# Export the sorted DataFrame to a CSV file
        final_amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [22]:
final_amazon_df

,title,price,rating,reviews,availability
1,"Samsung Galaxy M14 5G (Smoky Teal, 6GB, 128GB ...",,4.0 out of 5 stars,"8,734 ratings",In stock
8,"realme narzo N55 (Prime Blue, 6GB+128GB) 33W S...",,4.0 out of 5 stars,"8,117 ratings",In stock
14,"realme narzo N55 (Prime Black, 6GB+128GB) 33W ...",,4.0 out of 5 stars,"8,117 ratings",In stock
4,"Redmi 12C (Lavender Purple, 4GB RAM, 128GB Sto...",,3.9 out of 5 stars,"7,826 ratings",In stock
5,"realme narzo N53 (Feather Gold, 4GB+64GB) 33W ...",,3.9 out of 5 stars,"7,423 ratings",In stock
15,"realme narzo N53 (Feather Black, 4GB+64GB) 33W...",,3.9 out of 5 stars,"7,423 ratings",In stock
11,"Lava Blaze 5G (Glass Green, 4GB RAM, UFS 2.2 1...",,4.0 out of 5 stars,"5,121 ratings",In stock
9,"Oppo A17k (Blue, 3GB RAM, 64GB Storage) with N...",,4.0 out of 5 stars,380 ratings,In stock
7,"realme narzo 50i (Carbon Black, 4GB RAM+64GB S...",,4.0 out of 5 stars,"36,281 ratings",In stock
10,"OnePlus Nord CE 3 Lite 5G (Chromatic Gray, 8GB...",,4.1 out of 5 stars,"28,704 ratings",In stock
